In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

In [8]:
import os
import pickle
import warnings
from collections import defaultdict
from datetime import datetime

import numpy as np
import pandas as pd
from cobra.io import load_model, read_sbml_model
from matplotlib import pyplot as plt

from experiments.cue import plot_bge, plot_result
from experiments.experiment import Experiment
from experiments.fast_dFBA import (ConstantBounds, dFBA, make_bge_listener,
                                   make_boundary_listener, make_cue_listener,
                                   make_growth_rate_listener,
                                   make_shadow_price_listener,
                                   plot_shadow_prices, setup_drawdown)
from parameters.drawdown import *
from utils.cobra_utils import get_or_create_exchange
from utils.units import u

C_PER_GLUCOSE = 6
C_PER_ACETATE = 2

BIOMASS_ID = "Rpom_hwa_biomass"
DATA_FILE = "../data/clean/CUE2/dFBA.pkl"

# Load and set up model
model = read_sbml_model("../model/Rpom_05.xml")
setup_drawdown(model)
ex_ace = get_or_create_exchange(model, "ACET[e]")

# Turn on maintenance
atpm = model.reactions.get_by_id("ATPM")
atpm.bounds = (25, 25)

# Make ETC complex II irreversible, complex I reversible
model.reactions.get_by_id("RXN-14970-SUC/UBIQUINONE-8//FUM/CPD-9956.31.").bounds = (-1000, 0)
model.reactions.get_by_id("1.6.99.5-RXN-NADH/UBIQUINONE-10/PROTON//NAD/CPD-9958.40.").bounds = (-1000, 1000)
model.reactions.get_by_id("1.6.99.5-RXN-NADH/UBIQUINONE-6/PROTON//NAD/UBIQUINOL-30.43.").bounds = (-1000, 1000)
model.reactions.get_by_id("1.6.99.5-RXN-NADH/UBIQUINONE-9/PROTON//NAD/CPD-9957.39.").bounds = (-1000, 1000)

# Make 1.5.5.1-RXN-ETF-Reduced/UBIQUINONE-10//ETF-Oxidized/CPD-9958/PROTON.56. irreversible
model.reactions.get_by_id("1.5.5.1-RXN-ETF-Reduced/UBIQUINONE-10//ETF-Oxidized/CPD-9958/PROTON.56.").bounds = (0, 1000)

# Change/remove all reactions so as to use ubiquinone 10 only
model.remove_reactions([
    "1.5.5.1-RXN-ETF-Reduced/UBIQUINONE-6//ETF-Oxidized/UBIQUINOL-30/PROTON.59.",
    "1.5.5.1-RXN-ETF-Reduced/UBIQUINONE-9//ETF-Oxidized/CPD-9957/PROTON.55.",
    "1.6.99.5-RXN-NADH/UBIQUINONE-6/PROTON//NAD/UBIQUINOL-30.43.",
    "1.6.99.5-RXN-NADH/UBIQUINONE-9/PROTON//NAD/CPD-9957.39.",
    "2.1.1.64-RXN",
    "DHHB-METHYLTRANSFER-RXN",
    "RXN3O-102",
    "RXN66-550-ETF-Reduced/UBIQUINONE-8//ETF-Oxidized/CPD-9956/PROTON.55.",
    "SUCCINATE-DEHYDROGENASE-UBIQUINONE6-RXN"
])
# Add new ETC reactions
ubiquinones = [model.metabolites.get_by_id(ubi)
                for ubi in [
                    "UBIQUINONE-6[c]",
                    "UBIQUINONE-8[c]",
                    "UBIQUINONE-9[c]",
                    "UBIQUINONE-10[c]"]]
ubiquinols = [model.metabolites.get_by_id(ubi)
                for ubi in [
                    "UBIQUINOL-30[c]",
                    "CPD-9956[c]",
                    "CPD-9957[c]",
                    "CPD-9958[c]"
                ]]

reactions_to_change = [
    "RXN-14903-PRO/UBIQUINONE-8//L-DELTA1-PYRROLINE_5-CARBOXYLATE/CPD-9956/PROTON.67.",
    "RXN-14970-SUC/UBIQUINONE-8//FUM/CPD-9956.31.",
    "RXN-20992-CPD-9956/OXYGEN-MOLECULE//UBIQUINONE-8/WATER.45.",
]
for rxn_id in reactions_to_change:
    rxn = model.reactions.get_by_id(rxn_id)
    metabolites_copy = rxn.metabolites.copy()
    for met, coeff in metabolites_copy.items():
        if met in ubiquinones:
            rxn.add_metabolites({met : -coeff, model.metabolites.get_by_id("UBIQUINONE-10[c]") : coeff})
        if met in ubiquinols:
            rxn.add_metabolites({met : -coeff, model.metabolites.get_by_id("CPD-9958[c]") : coeff})

# Load condition data
with open(DATA_FILE, "rb") as f:
    data = pickle.load(f)

In [9]:
data.keys()

dict_keys([(<Quantity(2.0, 'millimolar')>, <Quantity(0.0, 'millimolar')>), (<Quantity(0.0, 'millimolar')>, <Quantity(6.0, 'millimolar')>), (<Quantity(0.666666667, 'millimolar')>, <Quantity(4.0, 'millimolar')>), (<Quantity(1.33333333, 'millimolar')>, <Quantity(2.0, 'millimolar')>)])

In [18]:
g12a0 = [v for (g, a), v in data.items() if g > 0 and a == 0][0]
g0a12 = [v for (g, a), v in data.items() if g == 0 and a > 0][0]
g4a8 = [v for (g, a), v in data.items() if 0 < g.magnitude < 1 and a > 0][0]
g8a4 = [v for (g, a), v in data.items() if g.magnitude > 1 and a > 0][0]

In [21]:
g12a0["raw"].keys()

dict_keys(['raw_b_t', 'raw_b', 'raw_g_t', 'raw_g_s', 'raw_a_t', 'raw_a_s'])